In [ ]:
import random
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from scipy.optimize import minimize
from util import (
    thermal_noise_watts, 
    freq_to_wavelength,
    normalDistribution
)

# Set font size
plt.rcParams["font.size"] = 12

# Seed random number generators
seed = 324
random.seed(seed)
np.random.seed(seed)
rng = np.random.default_rng(seed)

## Constant parameters for simulation


In [ ]:
r_min = 500                        # minimum flight path radius (meters)
r_max = 15000                      # maximum flight path radius (meters)
mu_x = 15000                       # x-mean of user distribution (meters)
mu_y = 0                           # y-mean of user distribution (meters)
sigma_x = 3000                     # x-standard dev. of distribution
sigma_y = 3000                     # y-standard dev. of distribution
rD = 2000                          # radius of user distribution (meters) - deprecated, should be std. dev.
cx_min = -20000                    # minimum x-coordinate of UAV (meters)
cx_max = 40000                     # maximum x-coordinate of UAV (meters)
cy_min = -20000                    # minimum y-coordinate of UAV (meters)
cy_max = 20000                     # maximum y-coordinate of UAV (meters)
v = 50                             # flight speed of UAV (meters/second)

N = 200                            # number of timeslots in one rotation
M = 2                              # number of users served per timeslot
G = 20                             # total number of users

# therefore, each user should get scheduled N*M/G times

H = 1000                           # altitude of UAV (meters)
PVtx = 10                          # UAV transmission power (watts)
PGtx = 0.01                        # user transmission power (watts)
Gamma_T = 1                        # antenna gain at transmitter
Gamma_R = 1                        # antenna gain at receiver
B = 1e6                            # channel bandwidth (Hz)
N0 = thermal_noise_watts(B)        # noise power spectral density (watts)
F = 2e9                            # communication frequnency (Hz)
WAVELENGTH = freq_to_wavelength(F) # communication wavelength (meters) 

## Precalculate angles and gather constants

In [ ]:
thetas = 2*np.pi*np.arange(N)/N                     # precalculate angles of UAV at each timeslot
cos_th = np.cos(thetas)                             # precalculate the cos of each UAV angle
sin_th = np.sin(thetas)                             # precalculate the sin of each UAV angle
CG = PGtx*Gamma_T*Gamma_R*(WAVELENGTH/(4*np.pi))**2 # gather constants 
CV = PVtx*Gamma_T*Gamma_R*(WAVELENGTH/(4*np.pi))**2 # gather constants
AG = (CG * M) / N0                                  # gather constants
AV = CV / N0                                        # gather constants

## User Generation
Here we define a function to randomly generate users within the deadzone. For the purposes of the simulation, we generate the maximum number of users. Smaller flight paths may not serve every user in one rotation, but the objective is normalized over timeslots. We plot the users at the end to verify that the distribution is correct.

In [ ]:
users = normalDistribution(G, mu_x, mu_y, sigma_x, sigma_y, seed=seed) # generate users from a normal distribution
print(f'len(users): {len(users)}')
print(f'G: {G}')

""" Generate a scatter plot to display the setup """
xs, ys = zip(*users)
plt.scatter(xs, ys, c='b', marker='o', s=5)
plt.scatter(0, 0, c='black', marker='2', s=200, label="Origin")
ax = plt.gca()
circle = patches.Circle((0, 0), radius=2000, fill=False, color='black', linewidth=2)
ax.add_patch(circle)
plt.grid(True)
plt.xlabel("x")
plt.ylabel("y")
plt.xlim(-10000, 10000)
plt.ylim(-10000, 10000)
plt.axis("equal")
plt.show()

## Mean SE (objective function)
The average achievable spectral efficiency of the relay system over the timeslots is given as
$$\overline{\text{SE}}(\alpha, r, \mathbf{x}^c, \mathcal{B}) = \frac{1}{N}\sum_{n=1}^N \min\left(\alpha\cdot \text{SE}^\text{GV}_n, (1-\alpha)\cdot \text{SE}^\text{VB}_n\right)$$

In [ ]:
def meanSE(alpha, 
           r, 
           cx, 
           cy, 
           B,
           users=users,
           AG=AG,
           AV=AV,
           H=H,
           M=M):

    # Compute UAV position at each timeslot
    vx = cx + r*cos_th
    vy = cy + r*sin_th

    # Compute squared distance from user to UAV
    dx = vx[:, None] - users[:, 0]
    dy = vy[:, None] - users[:, 1]
    dgv = dx*dx + dy*dy + H*H  

    # Compute squared distance from UAV to base station     
    dvb = vx*vx + vy*vy + H*H           

    # Calculate SEGV (user-to-uav spectral efficiency)
    log_terms = np.log2(1.0 + AG/dgv)
    segv = (alpha/M) * np.sum(B * log_terms, axis=1)

    # Calculate SEVB (uav-to-bs spectral effiency)
    sevb = (1-alpha) * np.log2(1.0 + AV/dvb)

    # Calculate the mins
    min_se = np.minimum(segv, sevb)

    # Return the average
    return np.mean(min_se)

## $\alpha$ Optimizer
Here I will define an optimizer to find the optimal $\alpha$ for a fixed $r$, $\mathbf{c}$.

In [ ]:
def optimize_alpha(r, 
                   cx, 
                   cy, 
                   B, 
                   users=users, 
                   AG=AG,
                   AV=AV,
                   H=H,
                   M=M,
                   verbose=False):
    
    # Compute UAV position at each timeslot
    vx = cx + r*cos_th
    vy = cy + r*sin_th

    # Compute squared distances from user to UAV
    dx = vx[:, None] - users[:, 0]
    dy = vy[:, None] - users[:, 1]
    dgv = dx*dx + dy*dy + H*H

    # Compute squared distances from UAV to BS
    dvb = vx*ax + vy*vy + H*H

    # Vectorized computation of segv and sevb
    log_terms = np.log2(1.0 + AG/dgv)      # Shape: (N, G)
    segv = np.sum(B * log_terms, axis=1)   # Shape: (N,) - sum over users for each timeslot
    sevb = np.log2(1.0 + AV/dvb)           # Shape: (N,)

    alpha_var = cp.Variable()  # cvxpy var representing timeshare
    eta_n_var = cp.Variable(N) # cvxpy var representing achievable se in each timeslot

    # Add constraints
    cons = []
    cons += [eta_n_var <= (alpha_var/M) * segv]
    cons += [eta_n_var <= (1 - alpha_var) * sevb]
    cons += [alpha_var >= 0, alpha_var <= 1]

    obj = cp.Maximize( cp.mean(eta_n_var) )

    prob = cp.Problem(obj, cons)
    prob.solve(solver=cp.MOSEK, verbose=verbose)

    if prob.status not in ("optimal", "optimal_inaccurate"):
        raise RuntimeError(f"SCA subproblem infeasible/failed: status {prob.status}")
    
    return float(alpha_var.value)

## $\mathcal{B}$ Optimizer

In [ ]:
def optimize_B(alpha, 
               r, 
               cx, 
               cy, 
               users=users,
               AG=AG,
               AV=AV,
               H=H,
               M=M,
               verbose=False):
    
    # Compute UAV position at each timeslot
    vx = cx + r*cos_th
    vy = cy + r*sin_th

    # Compute squared distances from user to UAV
    dx = vx[:, None] - users[:, 0]
    dy = vy[:, None] - users[:, 1]
    dgv = dx*dx + dy*dy + H*H

    # Compute squared distances from UAV to BS
    dvb = vx*ax + vy*vy + H*H

    # Calculate segv and sevb
    segv = (alpha/M) * np.log2(1.0 + AG/dgv)  # Shape: (N, num_users)
    sevb = (1-alpha) * np.log2(1.0 + AV/dvb)   # Shape: (N,)

    # define CVXPY variables
    b_vars = cp.Variable((N, G)) # per-user, per-timeslot scheduling
    eta_vars = cp.Variable(N)    # per-timeslot spectral efficiency

    # Constraints
    cons = []

    # Add eta_n constraints
    cons += [eta_vars[n] <= cp.sum(cp.multiply(b_vars[n], segv[n])) for n in range(N)]
    cons += [eta_vars <= sevb]
    
    # Box constraints on scheduling
    cons += [b_vars >= 0, b_vars <= 1]
    
    # Column sum constraint: each user at most N*M/G times
    cons += [cp.sum(b_vars, axis=0) <= (N*M)/G]
    
    # Row sum constraint: at most M users per timeslot
    cons += [cp.sum(b_vars, axis=1) <= M]

    # Want to maximize eta_n
    obj = cp.Maximize( cp.mean(eta_vars) )

    # Solve problem and check convergence
    prob = cp.Problem(obj, cons)
    prob.solve(solver=cp.MOSEK, verbose=verbose)

    if prob.status not in ("optimal", "optimal_inaccurate"):
        raise RuntimeError(f"SCA subproblem infeasible/failed: status {prob.status}")
    
    # Project back to binary
    # Find indices of the top-k elements per row
    idx = np.argpartition(b_vars.value, -M, axis=1)[:, -M:]  # unsorted top-k indices

    # Initialize output with zeros
    out = np.zeros_like(b_vars.value, dtype=int)

    # Set top-k positions to 1
    rows = np.arange(b_vars.shape[0])[:, None]
    out[rows, idx] = 1

    return out

## Random scheduling
random scheduling function for testing

In [ ]:
def random_schedule(M=M, maxiters=1000, tol=1e-9):
    mat = np.random.rand(N, G)

    for _ in range(maxiters):
        # Scale rows
        row_sums = mat.sum(axis=1, keepdims=True)
        mat *= (M / row_sums)

        # Scale columns
        col_sums = mat.sum(axis=0, keepdims=True)
        mat *= ( (N*M/G) / col_sums)

        # Check convergence
        if (np.allclose(mat.sum(axis=1), M, atol=tol) and
            np.allclose(mat.sum(axis=0), N, atol=tol)):
            break

    return mat

## Powell's optimizer for $\alpha$, $r$, $\mathbf{x}^c$
Here I define a block optimizer to iteratively optimize $\alpha$, $r$, $\mathbf{x}^c$.

In [ ]:
def objective(params, alpha, B, users, AG, AV, H, M):
    r, cx, cy = params
    return -meanSE(alpha, 
                   r, 
                   cx, 
                   cy, 
                   B, 
                   users=users,
                   AG=AG,
                   AV=AV,
                   H=H,
                   M=M)

def powells_optimizer(
                    alpha0,
                    r0, 
                    cx0, 
                    cy0, 
                    rbounds=(r_min, r_max),
                    cxbounds=(cx_min, cx_max),
                    cybounds=(cy_min, cy_max),
                    users=users,
                    AG=AG,
                    AV=AV,
                    H=H,
                    M=M,
                    tolerance=1e-3,
                    verbose=True
):

    # intial values for alpha, r, x^c
    alpha = alpha0
    r = r0
    cx = cx0
    cy = cy0

    # system bounds
    bounds = [rbounds, cxbounds, cybounds]

    # record history so we know when to stop
    traj_history = []
    obj_history = []

    it = 0
    while True:
        if verbose:
            print(f'Powell\'s Optimizer Iteration {it}')
            print(f'    Values at iteration {it}: alpha = {alpha}, r={r}, c=({cx}, {cy})')

        B = optimize_B(alpha, 
                       r, 
                       cx, 
                       cy, 
                       users=users,
                       AG=AG,
                       AV=AV,
                       H=H,
                       M=M
        )
        if verbose:
            print('    User scheduling optimized')

        alpha = optimize_alpha(r,
                               cx,
                               cy,
                               B,
                               users=users,
                               AG=AG,
                               AV=AV,
                               H=H,
                               M=M
        )
        if verbose:
            print('    Timeshare optimized')
        
        result = minimize(
                    objective,
                    [r,cx,cy],
                    args=(alpha, B, users, AG, AV, H, M),
                    method='Powell',
                    bounds=bounds,
                    options={
                        'maxiter':1000,
                        'xtol':1e-3,
                        'ftol':1e-3
                    }
        )
        if verbose:
            print(f'    Trajectory optimized, result.message: {result.message}\n')

        r, cx, cy = result.x
        obj_history.append(meanSE(alpha, 
                                  r, 
                                  cx, 
                                  cy, 
                                  B, 
                                  users=users,
                                  AG=AG,
                                  AV=AV,
                                  H=H,
                                  M=M))
        traj_history.append((r, cx, cy))

        if it > 0:
            if ( (obj_history[-1] - obj_history[-2])/obj_history[-2] < tolerance):
                break
        it += 1
    
    return alpha, B, traj_history
        
alpha0 = 0.5
r0 = (r_min + r_max) / 2
cx0 = (cx_min + cx_max) / 2
cy0 = (cy_min + cy_max) / 2

alpha_opt, b_opt, traj_hist = powells_optimizer(alpha0, r0, cx0, cy0)
r_opt, cx_opt, cy_opt = traj_hist[-1]

print(f'Optimal alpha: {alpha_opt}')
print(f'Optimal radius: {r_opt}')
print(f'Optimal center point: ({cx_opt}, {cy_opt})')
print(f'Achievable SE: {meanSE(alpha_opt, r_opt, cx_opt, cy_opt, b_opt)}')

xs, ys = zip(*users)
plt.scatter(xs, ys, c='b', marker='o', s=5)
plt.scatter(0, 0, c='black', marker='2', s=200, label="Origin")
plt.scatter(cx_opt, cy_opt+r_opt, c='r', marker='<', s=50, label="UAV")
ax = plt.gca()  # get current axes
coverage_zone = patches.Circle((0, 0), radius=2000, fill=False, color='black', linewidth=1)
flight_path = patches.Circle((cx_opt, cy_opt), radius=r_opt, fill=False, color='red', linewidth=1)

for traj in traj_hist:
    r, cx, cy = traj
    path = patches.Circle((cx, cy), radius=r, fill=False, color='blue', alpha=0.5, linewidth=1)
    ax.add_patch(path)

ax.add_patch(coverage_zone)
ax.add_patch(flight_path)
plt.grid(True)
plt.xlabel("x")
plt.ylabel("y")
plt.xlim(-10000, 10000)
plt.ylim(-10000, 10000)
plt.axis("equal")  # equal scaling
plt.show()


## Successive convex approximation

In [ ]:
def sca_optimizer(
                alpha0, 
                r0, 
                cx0, 
                cy0,
                r_bounds=(r_min, r_max), 
                cx_bounds=(cx_min, cx_max), 
                cy_bounds=(cy_min, cy_max),
                users=users,
                AG=AG,
                AV=AV,
                H=H,
                M=M,
                tol=2e-3,
                mosek_tol=1e-3,
                max_mosek_iters=30,
                verbose=False
):
    
    # initial values
    alpha = alpha0
    r = r0
    cx = cx0
    cy = cy0

    # Precompute constants
    users_x = users[:, 0]  # Shape: (G,)
    users_y = users[:, 1]  # Shape: (G,)
    H_squared = H * H

    # scale factor to help SCA converge
    scale_factor = 5e6
    grad_factor = scale_factor / np.log(2.0)

    def f_and_grad(s, A):
        """ Return f(s)=log2(1+A/s) and f'(s) = -A/(ln2*s*(s+A)) elementwise. """
        s = np.maximum(s, 1e-9)
        f = scale_factor * np.log2(1.0 + A/s)
        fp = -grad_factor * A / (s * (s + A))
        return f, fp
    
    traj_hist = []
    obj_hist = []
    dgv_error = []
    dvb_error = []

    it = 0
    while True:
        print(f'SCA Optimizer Iteration {it}')

        # user schedule optimization
        B = optimize_B(alpha,
                       r,
                       cx,
                       cy,
                       users=users,
                       AG=AG,
                       AV=AV,
                       H=H,
                       M=M)

        # timeshare optimization
        alpha = optimize_alpha(r,
                               cx,
                               cy, 
                               B, 
                               users=users,
                               AG=AG,
                               AV=AV,
                               H=H,
                               M=M)

        mosek_obj_hist = []

        # r, x^c optimization
        for mosek_it in range(max_mosek_iters):
            print(f'    MOSEG Iteration {mosek_it}')

            # Compute UAV coords
            vx = cx + r*cos_th
            vy = cy + r*sin_th

            # Compute squared distances
            # GV distance
            dx = vx[:, None] - users_x[None, :]  # Shape: (N, G)
            dy = vy[:, None] - users_y[None, :]  # Shape: (N, G)
            dgv = dx**2 + dy**2 + H_squared      # Shape: (N, G) 

            # VB distance
            dvb = vx*vx + vy*vy + H_squared           

            # Vectorized f and gradient evaluation
            fU0, gU0 = f_and_grad(dgv, AG)  # Both shape: (N, G)
            fB0, gB0 = f_and_grad(dvb, AV)  # Both shape: (N,)

            # CVXPY variables
            r_var  = cp.Variable()        # radius cvxpy var
            cx_var = cp.Variable()        # UAV x-pos cvxpy var
            cy_var = cp.Variable()        # UAV y-pos cvxpy var
            eta_var = cp.Variable(N)      # per-timeslot spectral efficiency
            dgv_var = cp.Variable((N, G)) # per-timeslot user-to-uav distance 
            dvb_var = cp.Variable(N)      # per-timeslot uav-to-bs distance

            cons = []

            vx_vars = cx_var + r_var * cos_th  # Length N vector of affine expressions
            vy_vars = cy_var + r_var * sin_th  # Length N vector of affine expressions

            # Geometric "upper-bound" constraints: s >= true squared distance
            for n in range(N):
                cons += [dgv_var[n, :] >= (vx_vars[n] - users_x)**2 + (vy_vars[n] - users_y)**2 + H_squared]
                cons += [dvb_var[n] >= vx_vars[n]**2 + vy_vars[n]**2 + H_squared]
            
            # Linearized (conservative) SE constraints per slot:
            # t_n <= (alpha/M) sum_k [ fU0 + gU0 * (s - s0) ]
            # t_n <= (1-alpha)      [ fB0 + gB0 * (sb - sb0) ]
            for n in range(N):
                # GV constraints
                linearized_seua = fU0[n, :] + cp.multiply(gU0[n, :], dgv_var[n, :] - dgv[n, :])
                weighted_seua = cp.multiply(B[n, :], linearized_seua)
                cons += [eta_var[n] <= (alpha / M) * cp.sum(weighted_seua)]

                # VB constraints
                cons += [eta_var[n] <= (1.0 - alpha) * (fB0[n] + gB0[n] * (dvb_var[n] - dvb[n]))]
            
            # Add parameter bounds
            r_min, r_max = r_bounds
            cx_min, cx_max = cx_bounds
            cy_min, cy_max = cy_bounds

            cons.extend([
                r_var >= r_min, r_var <= r_max,
                cx_var >= cx_min, cx_var <= cx_max,
                cy_var >= cy_min, cy_var <= cy_max
            ])

            # Solve objective
            obj = cp.Maximize( cp.mean(eta_var) )
            prob = cp.Problem(obj, cons)
            prob.solve(solver=cp.MOSEK, verbose=verbose)

            if prob.status not in ("optimal", "optimal_inaccurate"):
                raise RuntimeError(f"    MOSEK subproblem infeasible/failed at iter {mosek_it}: status {prob.status}")
            
            # Update iterate
            r = float(r_var.value)
            cx = float(cx_var.value)
            cy = float(cy_var.value)

            snk_it_error = 0
            sbn_it_error = 0
            for n in range(N):
                snk_it_error += np.sum(dgv_var.value[n, :] - ( (vx_vars.value[n] - users_x)**2 + (vy_vars.value[n] - users_y)**2 + H_squared ))
                sbn_it_error += dvb_var.value[n] - ( (vx_vars.value[n]**2 + vy_vars.value[n]**2 + H_squared) )
        
            snk_it_error /= N*G
            sbn_it_error /= N
            
            dgv_error.append(snk_it_error)
            dvb_error.append(sbn_it_error)

            traj_hist.append((r, cx, cy))
            mosek_obj_hist.append(float(np.mean(t.value)))

            if mosek_it > 0:
                if (mosek_obj_hist[-1] - mosek_obj_hist[-2])/max(1e-9, mosek_obj_hist[-2]) < mosek_tol:
                    break
        
        # Print iteration results
        print(f'SCA Iteration {it} results:')
        print(f'alpha = {alpha}, radius = {r}, center = ({cx}, {cy})')
        print()
        
        # Check for convergence
        obj_hist.append(meanSE(alpha,
                               r,
                               cx,
                               cy,
                               B,
                               users=users,
                               AG=AG,
                               AV=AV,
                               H=H,
                               M=M))
        if it > 0:
            if (obj_hist[-1] - obj_hist[-2])/max(1e-9, obj_hist[-2]) < tol:
                break
        
        it += 1
    
    return alpha, B, traj_hist, dgv_error, dvb_error

alpha0 = 0.5
r0 = (r_min + r_max) / 2
cx0 = (cx_min + cx_max) / 2
cy0 = (cy_min + cy_max) / 2

alpha_opt, b_opt, traj_hist, dgv_error, dvb_error = sca_optimizer(alpha0, r0, cx0, cy0)
r_opt, cx_opt, cy_opt = traj_hist[-1]

print(f'Optimal alpha: {alpha_opt}')
print(f'Optimal radius: {r_opt}')
print(f'Optimal center point: ({cx_opt}, {cy_opt})')
print(f'Achievable SE: {meanSE(alpha_opt, r_opt, cx_opt, cy_opt, b_opt)}')

# plt.plot(np.arange(0, len(dgv_error)), dgv_error, c='g', label='snk error')
# plt.plot(np.arange(0, len(dvb_error)), dvb_error, c='r', label='sbn error')
# plt.ylabel("Average error")
# plt.xlabel("Iteration")
# plt.legend()
# plt.grid(True)
# plt.show()

xs, ys = zip(*users)
plt.scatter(xs, ys, c='b', marker='o', s=5)
plt.scatter(0, 0, c='black', marker='2', s=200, label="Origin")
plt.scatter(cx_opt, cy_opt+r_opt, c='r', marker='<', s=50, label="UAV")
ax = plt.gca()  # get current axes
coverage_zone = patches.Circle((0, 0), radius=2000, fill=False, color='black', linewidth=1)
flight_path = patches.Circle((cx_opt, cy_opt), radius=r_opt, fill=False, color='red', linewidth=1)

for traj in traj_hist:
    r, cx, cy = traj
    path = patches.Circle((cx, cy), radius=r, fill=False, color='blue', alpha=0.5, linewidth=1)
    ax.add_patch(path)

ax.add_patch(coverage_zone)
ax.add_patch(flight_path)
plt.grid(True)
plt.xlabel("x")
plt.ylabel("y")
plt.xlim(-10000, 10000)
plt.ylim(-10000, 10000)
plt.axis("equal")  # equal scaling
plt.show()

# Failsafe SCA
Here I will define an optimization program which uses SCA when possible, but will revert to Powell's in the event of an exception

In [ ]:
def failsafe_optimize(
                alpha0, 
                r0, 
                cx0, 
                cy0,
                r_bounds=(r_min, r_max), 
                cx_bounds=(cx_min, cx_max), 
                cy_bounds=(cy_min, cy_max),
                users=users,
                AG=AG,
                AV=AV,
                H=H,
                M=M,
                tol=2e-3,
                mosek_tol=1e-3,
                max_mosek_iters=30,
                verbose=False
):
    try:
        print("Running SCA")
        alpha_opt, b_opt, traj_hist, dgv_error, dvb_error = sca_optimizer(
                                                                alpha0, 
                                                                r0, 
                                                                cx0, 
                                                                cy0,
                                                                r_bounds=r_bounds,
                                                                cx_bounds=cx_bounds,
                                                                cy_bounds=cy_bounds,
                                                                users=users,
                                                                AG=AG,
                                                                AV=AV,
                                                                H=H,
                                                                M=M,
                                                                tol=tol,
                                                                mosek_tol=mosek_tol,
                                                                max_mosek_iters=max_mosek_iters,
                                                                verbose=verbose)
    except:
        print("SCA failed - switching to Powell's")

        alpha_opt, b_opt, traj_hist = powells_optimizer(
                                                    alpha0,
                                                    r0, 
                                                    cx0, 
                                                    cy0, 
                                                    rbounds=(r_min, r_max),
                                                    cxbounds=(cx_min, cx_max),
                                                    cybounds=(cy_min, cy_max),
                                                    users=users,
                                                    AG=AG,
                                                    AV=AV,
                                                    H=H,
                                                    M=M,
                                                    tolerance=1e-3,
                                                    verbose=True)    
    finally:
        return alpha_opt, b_opt, traj_hist

alpha0 = 0.5
r0 = (r_min + r_max) / 2
cx0 = (cx_min + cx_max) / 2
cy0 = (cy_min + cy_max) / 2

alpha_opt, b_opt, traj_hist, dgv_error, dvb_error = sca_optimizer(alpha0, r0, cx0, cy0)
r_opt, cx_opt, cy_opt = traj_hist[-1]

print(f'Optimal alpha: {alpha_opt}')
print(f'Optimal radius: {r_opt}')
print(f'Optimal center point: ({cx_opt}, {cy_opt})')
print(f'Achievable SE: {meanSE(alpha_opt, r_opt, cx_opt, cy_opt, b_opt)}')

# plt.plot(np.arange(0, len(dgv_error)), dgv_error, c='g', label='snk error')
# plt.plot(np.arange(0, len(dvb_error)), dvb_error, c='r', label='sbn error')
# plt.ylabel("Average error")
# plt.xlabel("Iteration")
# plt.legend()
# plt.grid(True)
# plt.show()

xs, ys = zip(*users)
plt.scatter(xs, ys, c='b', marker='o', s=5)
plt.scatter(0, 0, c='black', marker='2', s=200, label="Origin")
plt.scatter(cx_opt, cy_opt+r_opt, c='r', marker='<', s=50, label="UAV")
ax = plt.gca()  # get current axes
coverage_zone = patches.Circle((0, 0), radius=2000, fill=False, color='black', linewidth=1)
flight_path = patches.Circle((cx_opt, cy_opt), radius=r_opt, fill=False, color='red', linewidth=1)

for traj in traj_hist:
    r, cx, cy = traj
    path = patches.Circle((cx, cy), radius=r, fill=False, color='blue', alpha=0.5, linewidth=1)
    ax.add_patch(path)

ax.add_patch(coverage_zone)
ax.add_patch(flight_path)
plt.grid(True)
plt.xlabel("x")
plt.ylabel("y")
plt.xlim(-10000, 10000)
plt.ylim(-10000, 10000)
plt.axis("equal")  # equal scaling
plt.show()